# Yahoo Finance

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz


In [2]:
frequency = "1d"
start_time = "2019-01-01"
end_time = (datetime.now(pytz.timezone('US/Pacific')) + timedelta(days=1)).strftime('%Y-%m-%d')

1. **TLT (iShares 20+ Year Treasury Bond ETF)**: Invests in U.S. Treasury bonds with remaining maturities greater than 20 years, aiming to track the investment results of an index composed of U.S. Treasury bonds.

2. **IEF (iShares 7-10 Year Treasury Bond ETF)**: Seeks to track the investment results of an index composed of U.S. Treasury bonds with remaining maturities between 7 and 10 years.

3. **EZU (iShares MSCI Eurozone ETF)**: Provides exposure to large and mid-sized companies in Eurozone countries, tracking the MSCI EMU Index.

4. **CNYA (iShares MSCI China A ETF)**: Offers exposure to large and mid-cap Chinese equities in the A-shares market, tracking the MSCI China A Inclusion Index.

5. **EWJ (iShares MSCI Japan ETF)**: Tracks the investment results of an index composed of Japanese equities, representing large and mid-sized companies in Japan.

6. **IGLT.L (iShares Core UK Gilts UCITS ETF)**: Invests in UK government bonds (gilts), seeking to track the performance of an index composed of sterling-denominated UK government bonds.

7. **XBB.TO (iShares Core Canadian Universe Bond Index ETF)**: Offers exposure to Canadian government and corporate bonds, tracking the performance of the broad Canadian bond market.

8. **BOND.AX (PIMCO Australian Bond Index Fund)**: Provides diversified exposure to the Australian bond market, including government, semi-government, and corporate debt securities.

9. **INDA (iShares MSCI India ETF)**: Aims to track the investment results of an index composed of Indian equities, representing large and mid-sized companies in India.

10. **EWL (iShares MSCI Switzerland ETF)**: Tracks the investment results of an index composed of Swiss equities, representing the Swiss stock market.

11. **ERUS (iShares MSCI Russia ETF)**: Seeks to track the investment results of an index composed of Russian equities.

12. **EWZ (iShares MSCI Brazil ETF)**: Aims to track the investment results of an index composed of Brazilian equities, reflecting the performance of the Brazilian stock market.

13. **EWY (iShares MSCI South Korea ETF)**: Tracks the investment results of an index composed of South Korean equities, representing the South Korean stock market.

14. **ENZL (iShares MSCI New Zealand ETF)**: Seeks to track the investment results of an index composed of New Zealand equities.

15. **EWD (iShares MSCI Sweden ETF)**: Aims to track the investment results of an index composed of Swedish equities, representing the Swedish stock market.

16. **EWM (iShares MSCI Malaysia ETF)**: Seeks to track the investment results of an index composed of Malaysian equities, representing the Malaysian stock market.

In [3]:
import yfinance as yf

# Define ETFs
etfs = ["TLT", "IEF", "EZU", "CNYA", "EWJ", "IGLT.L", "XBB.TO", "BOND.AX", "INDA", "EWL", "ERUS", "EWZ", "EWY", "ENZL", "EWD", "EWM"]

# Fetch ETF data
etf_data = {etf: yf.Ticker(etf).info for etf in etfs}

# Retrieve AUM or market cap
aum = {etf: data.get('totalAssets', 0) for etf, data in etf_data.items()}  # Replace 'totalAssets' with the desired metric

# Calculate total AUM
total_aum = sum(aum.values())

# Calculate weightings
weights = {etf: value / total_aum for etf, value in aum.items()}

print(weights)


{'TLT': 0.39267618391888776, 'IEF': 0.22443656602581535, 'EZU': 0.055452796117698704, 'CNYA': 0.001375688384817312, 'EWJ': 0.11945573685048876, 'IGLT.L': 0.0, 'XBB.TO': 0.05190119913096608, 'BOND.AX': 0.0002918638340707321, 'INDA': 0.06344915496245672, 'EWL': 0.009694608683074311, 'ERUS': 3.605101386706796e-06, 'EWZ': 0.04410063190258315, 'EWY': 0.03234883481068534, 'ENZL': 0.0008580180075443223, 'EWD': 0.0020872262553342084, 'EWM': 0.0018678860141905214}


In [4]:
# Existing tickers and additional ETF tickers
tickers = ["ETH-CAD", "BTC-CAD", "ETH-BTC", "SPY", "QQQ", "^IRX", "^TNX", "^TYX"]
additional_tickers = etfs  # ETFs list
tickers.extend(additional_tickers)

# Retrieve historical data
historical_data = {}
for ticker in tickers:
    historical_data[ticker] = yf.download(ticker, start=start_time, end=end_time, interval=frequency)[['High', 'Low']]

# Combine and weight data
combined_data = pd.DataFrame()
for ticker, data in historical_data.items():
    for col in data.columns:
        combined_column_name = f"{ticker}_{col}"  # Create a unique column name for each 'High' and 'Low'
        if ticker in weights:  # Apply weighting for ETFs
            combined_data[combined_column_name] = data[col] * weights[ticker]
        else:  # Use the data as is for other tickers
            combined_data[combined_column_name] = data[col]

# Calculate the Global Liquidity Index as the sum of all (weighted and non-weighted) columns
combined_data['Global_Liquidity_Index'] = combined_data.sum(axis=1)
combined_data.fillna(method='ffill', inplace=True)
# combined_data now contains the weighted Global Liquidity Index and individual ticker data


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [5]:
# Calculate simple moving averages for the 'ETH-CAD_High' and 'ETH-CAD_Low' columns
sma_periods = [5, 10, 20, 50, 100]  # Define the periods for the moving averages

for period in sma_periods:
    # Calculate the SMA for 'ETH-CAD_High'
    combined_data[f'ETH-CAD_High_SMA_{period}'] = combined_data['ETH-CAD_High'].rolling(window=period).mean()

    # Calculate the SMA for 'ETH-CAD_Low'
    combined_data[f'ETH-CAD_Low_SMA_{period}'] = combined_data['ETH-CAD_Low'].rolling(window=period).mean()

# combined_data now contains the SMA trend columns for 'ETH-CAD_High' and 'ETH-CAD_Low' for 5, 10, and 20-day periods

In [6]:
#combined_data.dropna(inplace=True)
combined_data

,ETH-CAD_High,ETH-CAD_Low,BTC-CAD_High,BTC-CAD_Low,ETH-BTC_High,ETH-BTC_Low,SPY_High,SPY_Low,QQQ_High,QQQ_Low,...,ETH-CAD_High_SMA_5,ETH-CAD_Low_SMA_5,ETH-CAD_High_SMA_10,ETH-CAD_Low_SMA_10,ETH-CAD_High_SMA_20,ETH-CAD_Low_SMA_20,ETH-CAD_High_SMA_50,ETH-CAD_Low_SMA_50,ETH-CAD_High_SMA_100,ETH-CAD_Low_SMA_100
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,192.649857,180.575409,5246.870117,5057.234375,0.036718,0.035544,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,213.886246,191.795868,5388.129883,5191.100586,0.039858,0.036676,251.210007,245.949997,155.750000,150.880005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,212.573837,198.451218,5362.917969,5159.898926,0.039754,0.038425,248.570007,243.669998,153.259995,149.490005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,211.151260,199.472687,5172.040527,5087.920898,0.040691,0.038730,253.110001,247.169998,157.000000,151.740005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,215.118729,206.441116,5223.182617,5131.535645,0.041418,0.040067,253.110001,247.169998,157.000000,151.740005,...,209.075986,195.347260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,4208.451172,4024.901611,70131.234375,69194.062500,0.060071,0.057951,508.750000,505.859985,438.600006,436.359985,...,4092.378076,3936.985840,4012.494800,3855.823682,3749.960229,3610.341797,3456.048413,3323.479966,3239.466746,3114.996196
2024-02-27,4320.133301,4108.095215,74229.710938,68871.843750,0.058200,0.059628,507.160004,504.750000,437.989990,435.029999,...,4140.318115,3980.789844,4059.198462,3894.273657,3804.805969,3660.229858,3482.079678,3346.548916,3255.612717,3129.713450
2024-02-28,4434.617188,4279.466797,77857.093750,73603.468750,0.057435,0.058136,506.859985,504.959991,436.670013,434.299988,...,4209.855176,4052.492188,4124.278906,3954.730225,3862.075220,3715.518799,3507.785601,3374.083032,3272.309731,3145.817271


In [7]:
combined_data.to_parquet('yahoo-finance.parquet')